# 3장. Train Model
이번 챕터에서는 수집한 이미지를 받아들여 목표에 해당하는 일련의 x, y 값 집합을 출력하는 신경망을 훈련할 것입니다.  
Road following을 위해 ResNet18 신경망 아키텍처 모델을 훈련하기 위해 PyTorch 딥러닝 프레임워크를 사용할 것입니다.  

이전 챕터에서 ‘dataset_xy_test’ 에 저장한 데이터를 갖고 model을 학습할 예정입니다.  
다만, 외부에서 데이터를 수집해서 갖고 오고자 하는 경우, data 파일들을 압축해서 현재 챕터에서 사용할 노트북 파일과 같은 경로에 업로드 해주시길 바랍니다.

## 외부 파일 가져오기
아래 명령어를 사용해서 압축을 풀어줍니다.  파일명에 주의합니다.

In [ ]:
!unzip -q road_following.zip

## 라이브러리 가져오기

In [1]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np

## DataSet 인스턴스 생성하기
여기에서는 torch.utils.data.Dataset 클래스를 구현하는 사용자 정의 클래스를 생성합니다.  
이 클래스는 len 및 getitem 함수를 구현합니다. 이 클래스는 이미지를 로드하고 이미지 파일 이름에서 x, y 값을 파싱하는 역할을 합니다.   
torch.utils.data.Dataset 클래스를 구현했으므로 torch 데이터 유틸리티를 모두 사용할 수 있습니다.  

우리는 데이터셋에 일부 변환(예: 색상 변형)을 하드 코딩했습니다.  
우리는 무작위 수평 뒤집기를 선택 사항으로 만들었습니다(인공지능 무인운반차량(AGV)이 '오른쪽에 머무르는' 도로와 같이 비대칭 적인 경로를 따라가고 싶을 때).  
인공지능 무인운반차량(AGV)이 어떤 관습을 따르는 지 여부가 중요하지 않은 경우, 뒤집기를 활성화하여 데이터셋을 증강할 수 있습니다.

In [2]:
DATASET_DIR = 'dataset_xy_test1'

#image 이름으로 저장된 x 값을 읽어 오는 함수
def get_x(path):
    """Gets the x value from the image filename"""
    return (float(int(path[3:6])) - 50.0) / 50.0

#image 이름으로 저장된 y 값을 읽어 오는 함수
def get_y(path):
    """Gets the y value from the image filename"""
    return (float(int(path[7:10])) - 50.0) / 50.0


class XYDataset(torch.utils.data.Dataset):
    def __init__(self, directory, random_hflips=False):
        self.directory = directory
        self.random_hflips = random_hflips
        self.image_paths = glob.glob(os.path.join(self.directory, '*.jpg'))
        self.color_jitter = transforms.ColorJitter(0.3, 0.3, 0.3, 0.3)
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        
        image = PIL.Image.open(image_path)
        x = float(get_x(os.path.basename(image_path)))
        y = float(get_y(os.path.basename(image_path)))
        
        if float(np.random.rand(1)) > 0.5:
            image = transforms.functional.hflip(image)
            x = -x
        
        image = self.color_jitter(image)
        image = transforms.functional.resize(image, (224, 224))
        image = transforms.functional.to_tensor(image)
        image = image.numpy()[::-1].copy()
        image = torch.from_numpy(image)
        image = transforms.functional.normalize(image, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
        return image, torch.tensor([x, y]).float()
    
dataset = XYDataset(DATASET_DIR, random_hflips=False)

## DataSet 분할하기
데이터셋을 읽은 후에는 데이터셋을 훈련 세트와 테스트 세트로 분할할 것입니다.  
이 예에서는 훈련 세트와 테스트 세트를 90%-10%로 분할합니다.   
테스트 세트는 훈련한 모델의 정확도를 검증하는 데 사용될 것입니다.

In [3]:
test_percent = 0.1
num_test = int(test_percent * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - num_test, num_test])

## DataLoader 생성하기
데이터를 일괄 처리로 로드하기 위해 DataLoader 클래스를 사용하여 데이터 로더를 생성합니다.  
이를 통해 데이터를 일괄 처리로 로드하고 데이터를 섞고, 여러 개의 서브프로세스를 사용할 수 있습니다.  
이 예에서는 배치 크기를 64로 사용합니다.  
배치 크기는 GPU의 사용 가능한 메모리에 따라 결정되며 모델의 정확도에 영향을 줄 수 있습니다.

In [4]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

## model 정의하기

PyTorch TorchVision에서 제공하는 ResNet-18 모델을 사용합니다.

In [5]:
model = models.resnet18(pretrained=True)

/mnt/c/jupyter ex/venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/mnt/c/jupyter ex/venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


전이 학습(transfer learning)이라는 프로세스에서, 수백만 장의 이미지로 훈련된 사전 훈련된 모델을 다시 사용하여 가능한 매우 적은 데이터로 이루어진 새로운 작업에 활용할 수 있습니다.  
- ResNet-18 상세설명 : https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py 
- 전이학습에 대한 유투브 설명: https://www.youtube.com/watch?v=yofjFQddwHE 


기본적으로, ResNet 모델은 fully connected (fc) 최종 레이어를 가지고 있으며, 입력 특성 수로 512를, 회귀를 위해 출력 특성 수로 1을 사용할 것입니다.  
하지만, 우리는 x,y 두 개의 값을 도출해야 하기 때문에, 마지막 은닉층에 레이어를 하나 추가해서, 2개의 output 데이터가 나오도록 할 예정입니다.  

마지막으로, 모델을 GPU에서 실행할 수 있도록 전송합니다.

In [6]:
model.fc = torch.nn.Linear(512, 2)
device = torch.device('cuda')
model = model.to(device)

## model 훈련하기
손실이 감소되면 최상의 모델을 저장하기 위해 50 에포크 동안 훈련합니다.  
훈련을 모두 마치면, "Success" 가 출력됩니다.

In [8]:
NUM_EPOCHS = 300
BEST_MODEL_PATH = 'best_steering_model_xy_test_12_13_1.pth'
best_loss = 1e9

optimizer = optim.Adam(model.parameters())

for epoch in range(NUM_EPOCHS):
    
    model.train()
    train_loss = 0.0
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        train_loss += float(loss)
        loss.backward()
        optimizer.step()
    train_loss /= len(train_loader)
    
    model.eval()
    test_loss = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        test_loss += float(loss)
    test_loss /= len(test_loader)
    
    print('%f, %f' % (train_loss, test_loss))
    if test_loss < best_loss:
        #colab에서 model을 학습할 경우 아래 옵션을 추가한 코드를 실행해야 한다.
        #torch.save(model.state_dict(), BEST_MODEL_PATH,_use_new_zipfile_serialization=False)
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_loss = test_loss
print('success')


/tmp/ipykernel_897/4253364535.py:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if float(np.random.rand(1)) > 0.5:
/tmp/ipykernel_897/4253364535.py:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if float(np.random.rand(1)) > 0.5:
/tmp/ipykernel_897/4253364535.py:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if float(np.random.rand(1)) > 0.5:
/tmp/ipykernel_897/4253364535.py:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprec

0.137653, 35.263405


/tmp/ipykernel_897/4253364535.py:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if float(np.random.rand(1)) > 0.5:
/tmp/ipykernel_897/4253364535.py:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if float(np.random.rand(1)) > 0.5:
/tmp/ipykernel_897/4253364535.py:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if float(np.random.rand(1)) > 0.5:
/tmp/ipykernel_897/4253364535.py:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprec

0.016937, 0.284504


/tmp/ipykernel_897/4253364535.py:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if float(np.random.rand(1)) > 0.5:
/tmp/ipykernel_897/4253364535.py:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if float(np.random.rand(1)) > 0.5:
/tmp/ipykernel_897/4253364535.py:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if float(np.random.rand(1)) > 0.5:
/tmp/ipykernel_897/4253364535.py:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprec

...

0.017956, 0.018849


/tmp/ipykernel_897/4253364535.py:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if float(np.random.rand(1)) > 0.5:
/tmp/ipykernel_897/4253364535.py:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if float(np.random.rand(1)) > 0.5:
/tmp/ipykernel_897/4253364535.py:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if float(np.random.rand(1)) > 0.5:
/tmp/ipykernel_897/4253364535.py:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprec

0.011643, 0.019449


/tmp/ipykernel_897/4253364535.py:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if float(np.random.rand(1)) > 0.5:
/tmp/ipykernel_897/4253364535.py:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if float(np.random.rand(1)) > 0.5:
/tmp/ipykernel_897/4253364535.py:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if float(np.random.rand(1)) > 0.5:
/tmp/ipykernel_897/4253364535.py:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprec

0.006031, 0.013378
success


모델이 훈련되면 best_steering_model_xy.pth 파일이 생성됩니다.  
이 파일은 다음 챕터인 Live Demo 노트북에서 추론에 사용할 수 있습니다.  
인공지능 무인운반차량(AGV) 이외의 다른 기기에서 훈련했다면, 훈련된 model 파일을 road_following 예제 폴더와 같은 경로로 인공지능 무인운반차량(AGV)에 업로드해야 합니다.